In [1]:
import time
from datetime import datetime
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [4]:
df = sqlContext.read.format('com.databricks.spark.csv').\
options(header='true', inferschema='true').\
    load('file:///home/oxclo/datafiles/wind/*')

In [5]:
df.first()

Row(Station_ID='SF15', Station_Name='Warnerville Switchyard, Oakdale', Location_Label='Warnerville', Interval_Minutes=5, Interval_End_Time='2015-01-5? 00:05', Wind_Velocity_Mtr_Sec=1.628, Wind_Direction_Variance_Deg=8.1, Wind_Direction_Deg=148.5, Ambient_Temperature_Deg_C=0.92, Global_Horizontal_Irradiance=0.061)

In [12]:
convertTime = lambda t: datetime.fromtimestamp(time.mktime(time.strptime(t, "%Y-%m-%d? %H:%M")))                      

In [13]:
toRow = lambda s: \
Row(stationid=s.Station_ID, \
time=convertTime(s.Interval_End_Time), \
direction=s.Wind_Direction_Deg, \
temp=s.Ambient_Temperature_Deg_C, \
velocity=s.Wind_Velocity_Mtr_Sec)

In [14]:
newDF = df.rdd.map(toRow).toDF()

In [21]:
newDF.write.format("org.apache.spark.sql.cassandra").mode('append').options(table="winddate", keyspace="wind").save()